In [1]:
import os
import omegaconf
from shell.utils.experiment_utils import *
from shell.utils.metric import *
import matplotlib.pyplot as plt
from shell.fleet.network import TopologyGenerator
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
import numpy as np

In [2]:
save_root_dir = "debug_replay_results"
dataset = "mnist"
# algo = "monolithic"
algo = "modular"
num_train = 64
seed = 0
use_contrastive = True
# use_contrastive = False

In [3]:
save_root_dir = "debug_replay_results"
dataset = "cifar100"
# algo = "monolithic"
algo = "modular"
num_train = 256
seed = 0
use_contrastive = False

In [4]:
job_name = f"{dataset}_{algo}_numtrain_{num_train}"
if use_contrastive:
    job_name += "_contrastive"
experiment = os.path.join(save_root_dir, job_name, dataset,algo, f"seed_{seed}")

In [5]:
config_path = os.path.join(experiment, "hydra_out", ".hydra", "config.yaml")
# read the config file
cfg = omegaconf.OmegaConf.load(config_path)
cfg

{'train': {'component_update_freq': 2, 'num_epochs': 2, 'init_component_update_freq': 2, 'init_num_epochs': 2, 'save_freq': 20}, 'dataset': {'dataset_name': 'cifar100', 'num_tasks': 5, 'num_classes_per_task': 5, 'with_replacement': False, 'num_trains_per_class': 256, 'num_vals_per_class': -1, 'remap_labels': True}, 'net': {'name': 'cnn', 'depth': 4, 'channels': 50, 'conv_kernel': 3, 'maxpool_kernel': 2, 'padding': 1, 'dropout': 0.5}, 'sharing_strategy': {'name': 'no_sharing', 'num_coms_per_round': 0}, 'seed': 0, 'algo': 'modular', 'job_name': 'cifar100_modular_numtrain_256', 'num_agents': 2, 'root_save_dir': 'debug_replay_results', 'parallel': True, 'num_init_tasks': 4, 'agent': {'save_dir': '${root_save_dir}/${job_name}/${dataset.dataset_name}/${algo}/seed_${seed}', 'batch_size': 64, 'memory_size': 32, 'improvement_threshold': 0.05, 'use_contrastive': False}}

In [6]:
graph, datasets, NetCls, LearnerCls, net_cfg, agent_cfg, train_cfg = setup_experiment(cfg)
len(datasets)

{'train': {'component_update_freq': 2, 'num_epochs': 2, 'init_component_update_freq': 2, 'init_num_epochs': 2, 'save_freq': 20}, 'dataset': {'dataset_name': 'cifar100', 'num_tasks': 5, 'num_classes_per_task': 5, 'with_replacement': False, 'num_trains_per_class': 256, 'num_vals_per_class': -1, 'remap_labels': True}, 'net': {'name': 'cnn', 'depth': 4, 'channels': 50, 'conv_kernel': 3, 'maxpool_kernel': 2, 'padding': 1, 'dropout': 0.5}, 'sharing_strategy': {'name': 'no_sharing', 'num_coms_per_round': 0}, 'seed': 0, 'algo': 'modular', 'job_name': 'cifar100_modular_numtrain_256', 'num_agents': 2, 'root_save_dir': 'debug_replay_results', 'parallel': True, 'num_init_tasks': 4, 'agent': {'save_dir': '${root_save_dir}/${job_name}/${dataset.dataset_name}/${algo}/seed_${seed}', 'batch_size': 64, 'memory_size': 32, 'improvement_threshold': 0.05, 'use_contrastive': False}}
i_size 32
num_classes 5
net_cfg {'name': 'cnn', 'depth': 4, 'channels': 50, 'conv_kernel': 3, 'maxpool_kernel': 2, 'padding': 1

2

In [7]:
classes_sequence_list = [dataset.class_sequence for dataset in datasets]
classes_sequence_list

[array([97, 66, 33, 19, 34,  3, 12,  2, 13, 92, 80,  5, 27, 60, 36, 11,  9,
        25, 31, 87, 58, 44, 94, 95, 64, 61, 62, 86, 79, 98, 93, 51, 22, 59,
        26, 37, 83, 73, 78,  1, 30, 55, 85, 47, 41, 20, 63, 43, 90, 88, 21,
        40, 15, 75,  4,  0, 52, 23, 35, 53, 84, 69, 16, 10, 71, 81,  6, 91,
        56, 76, 38, 50, 57, 74, 49, 18, 24, 39, 48, 99,  8, 65, 89, 67, 54,
        32, 42, 14, 68, 45, 96, 77, 46, 28, 72, 17, 29, 82,  7, 70]),
 array([13, 55, 29, 67, 64, 47, 58, 19, 30, 38, 15, 16, 34, 35, 73, 96, 25,
        89, 63,  5, 44, 21, 28, 60, 80, 86, 36, 95, 83, 40,  2, 24, 37, 61,
        62, 57, 53, 79, 27, 82, 65,  4, 22, 45, 78, 33, 32,  9, 90, 56, 46,
        48,  7, 59, 41, 93, 98, 69, 51, 74, 68, 72, 71, 39, 54, 18, 91, 76,
        70, 11, 10, 99, 77, 52, 87, 31,  6,  1, 84, 43, 85, 42, 12, 94, 26,
        88, 17, 49, 75, 92,  8, 97, 66, 50,  0, 23, 20, 81, 14,  3])]

In [8]:
task_id = 4
agent_id = 0
num_added_components = None
net = load_net(cfg, NetCls, net_cfg, agent_id=agent_id, task_id=task_id, num_added_components=num_added_components)
net

CNNSoftLLDynamic(
  (structure): ParameterList(
      (0): Parameter containing: [torch.float32 of size 4x4 (GPU 0)]
      (1): Parameter containing: [torch.float32 of size 4x4 (GPU 0)]
      (2): Parameter containing: [torch.float32 of size 4x4 (GPU 0)]
      (3): Parameter containing: [torch.float32 of size 4x4 (GPU 0)]
      (4): Parameter containing: [torch.float32 of size 4x4 (GPU 0)]
  )
  (softmax): Softmax(dim=0)
  (components): ModuleList(
    (0-3): 4 x Conv2d(50, 50, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  )
  (relu): ReLU()
  (maxpool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (dropout): Dropout(p=0.5, inplace=False)
  (decoder): ModuleList(
    (0-4): 5 x Linear(in_features=200, out_features=5, bias=True)
  )
  (transform): Normalize(mean=(0.5079, 0.4872, 0.4415), std=(0.2676, 0.2567, 0.2765))
  (projector): Sequential(
    (0): Linear(in_features=200, out_features=200, bias=True)
    (1): ReLU(inplace=True)
    (2): Linear(

In [9]:
dataset = datasets[agent_id]
testloaders = {task: torch.utils.data.DataLoader(testset,
                                                         batch_size=128,
                                                         shuffle=False,
                                                         num_workers=0,
                                                         pin_memory=True,
                                                         ) for task, testset in enumerate(dataset.testset[:(task_id+1)])}
# testloaders = {task: torch.utils.data.DataLoader(testset,
#                                                          batch_size=128,
#                                                          shuffle=False,
#                                                          num_workers=0,
#                                                          pin_memory=True,
#                                                          ) for task, testset in enumerate(dataset.valset[:(task_id+1)])}
testloaders

{0: <torch.utils.data.dataloader.DataLoader at 0x7f4436e3ab00>,
 1: <torch.utils.data.dataloader.DataLoader at 0x7f4436e3b760>,
 2: <torch.utils.data.dataloader.DataLoader at 0x7f4436e3b5e0>,
 3: <torch.utils.data.dataloader.DataLoader at 0x7f4436e3bd00>,
 4: <torch.utils.data.dataloader.DataLoader at 0x7f4436e3bbb0>}

In [26]:
acc = {}
net.eval()
with torch.no_grad():
    for task_id, testloader in testloaders.items():
        correct = 0.
        n = 0.
        for X, y in testloader:
            X = X.to(net.device)
            y = y.to(net.device)
            Y_hat = net(X, task_id)
            correct += (Y_hat.argmax(dim=1) == y).sum().item()
            n += len(y)
            acc[task_id] = correct/n

acc["avg"] = sum(acc.values())/len(acc)
acc

{0: 0.308, 1: 0.482, 2: 0.474, 3: 0.332, 4: 0.21, 'avg': 0.3612}

In [11]:
net.structure[1]

Parameter containing:
tensor([[-inf, -inf, -inf, -inf],
        [1., 1., 1., 1.],
        [-inf, -inf, -inf, -inf],
        [-inf, -inf, -inf, -inf]], device='cuda:0', requires_grad=True)

In [ ]:
X_out = [] # features
y_out = [] # global labels
y_task = [] # globallabel_task_id
with torch.no_grad():
    for task_id, testloader in testloaders.items():
        for X, y in testloader:
            X = X.to(net.device)
            X_encode = net.contrastive_embedding(X, task_id)
            X_out.append(X_encode.cpu())
            y_out.append(y.cpu())
            y_task.append(np.ones_like(y) * task_id)
X_encode = np.concatenate(X_out, axis=0)
Y = np.concatenate(y_out, axis=0)
y_task = np.concatenate(y_task, axis=0)

In [ ]:
X_encode.shape

In [ ]:
print(dataset.class_sequence)
print(np.unique(Y, return_counts=True))

In [ ]:
X_embedded = TSNE(n_components=2, random_state=0, init="pca", n_jobs=-1).fit_transform(X_encode)

In [ ]:
plt.figure(figsize=(20,10))

for y_label in np.unique(Y):
    plt.scatter(X_embedded[Y == y_label, 0], X_embedded[Y == y_label, 1], label=y_label)

plt.legend();

In [ ]:
X_embedded.shape

In [ ]:
# create an array same size as y and y_task where each element is {y}_{y_task} string
y_task_str = np.array([str(Y[i]) + "_" + str(y_task[i]) for i in range(len(Y))]) # class_task
# plot X_embedded with color corresponding to y_task_str
import seaborn as sns
# different sns color palette
# bigger plot size
sns.set(rc={'figure.figsize':(20,10)})
sns.set_palette("tab20")

task = 0
X_t = X_embedded[y_task == task]
y_t = y_task_str[y_task == task]
sns.scatterplot(x=X_t[:, 0], y=X_t[:, 1], hue=y_t);
# sns.scatterplot(x=X_embedded[:, 0], y=X_embedded[:, 1], hue=y_task_str);